<h1>CMYK2Lab - 含PS製作CMYKLab資料</h1>

## Library

In [1]:
import pandas as pd
import numpy as np
from win32com.client import Dispatch
from photoshop import Session
import random
from tqdm import tqdm
import time
from comtypes.client import GetActiveObject
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

## Photoshop CMYK 轉 Lab

#### Training Data

In [ ]:
CMYK_Lab = {'C':[],'M':[],'Y':[],'K':[],'L':[],'a':[],'b':[]}

app = Dispatch("Photoshop.Application")

with Session() as ps: #開新檔案
    ps.app.preferences.rulerUnits = ps.Units.Pixels
    ps.app.documents.add(1920, 1080, name="my_new_document")
    #文字顏色
    doc = ps.active_document
    new_text_layer = doc.artLayers.add()  #新增圖層
    new_text_layer.kind = ps.LayerKind.TextLayer #指定為字圖層
    new_text_layer.textItem.contents = "Hello, World!" #打字
    new_text_layer.textItem.size = 100
    textColor = ps.SolidColor()
    for c in range(0,120,20):
        textColor.cmyk.cyan= c
        for m in range(0,120,20):
            textColor.cmyk.magenta= m
            for y in range(0,120,20):
                textColor.cmyk.yellow = y
                for k in range(0,120,20):
                    textColor.cmyk.black = k
                    #命令更換文字顏色
                    new_text_layer.textItem.color = textColor
                    L = np.round(new_text_layer.textItem.color.Lab.L,0)
                    a = np.round(new_text_layer.textItem.color.Lab.a +0.5,0)
                    b = np.round(new_text_layer.textItem.color.Lab.b +0.5,0)
                    CMYK_Lab['C'].append(c), CMYK_Lab['M'].append(m), CMYK_Lab['Y'].append(y)
                    CMYK_Lab['K'].append(k)
                    CMYK_Lab['L'].append(L), CMYK_Lab['a'].append(a), CMYK_Lab['b'].append(b)
                    time.sleep(randint(0,2))
                    print(len(CMYK_Lab['C']),end="")
CMYK_Lab = pd.DataFrame(CMYK_Lab)
CMYK_Lab.to_csv('TrainDataSet_int.csv',index=False)

In [4]:
"""
為套用Color Profile，需先設立好指定ICC的檔案再開啟
"""
CMYK_Lab = {'C':[],'M':[],'Y':[],'K':[],'L':[],'a':[],'b':[]}

app = GetActiveObject("Photoshop.Application")


file = 'C:\python3.7\Scripts\programs\Fogra51.psd'
app.open(file)

with Session() as ps:     #文字顏色
    doc = ps.active_document
    new_text_layer = doc.artLayers.add()  #新增圖層
    new_text_layer.kind = ps.LayerKind.TextLayer #指定為字圖層
    new_text_layer.textItem.contents = "Hello, World!" #打字
    new_text_layer.textItem.size = 100
    textColor = ps.SolidColor()
    for c in range(0,120,20):
        textColor.cmyk.cyan= c
        for m in range(0,120,20):
            textColor.cmyk.magenta= m
            for y in range(0,120,20):
                textColor.cmyk.yellow = y
                for k in range(0,120,20):
                    textColor.cmyk.black = k
                    #命令更換文字顏色
                    new_text_layer.textItem.color = textColor
                    L = np.round(new_text_layer.textItem.color.Lab.L,0)
                    a = np.round(new_text_layer.textItem.color.Lab.a +0.5,0)
                    b = np.round(new_text_layer.textItem.color.Lab.b +0.5,0)
                    CMYK_Lab['C'].append(c), CMYK_Lab['M'].append(m), CMYK_Lab['Y'].append(y)
                    CMYK_Lab['K'].append(k)
                    CMYK_Lab['L'].append(L), CMYK_Lab['a'].append(a), CMYK_Lab['b'].append(b)
                    time.sleep(random.randint(0,2))
                    print(len(CMYK_Lab['C']),end="")
CMYK_Lab = pd.DataFrame(CMYK_Lab)
CMYK_Lab.to_csv('FograTrainDataSet.csv',index=False)

1234567891011121314151617181920212223242526272829303132333435363738394041424344454647484950515253545556575859606162636465666768697071727374757677787980818283848586878889909192939495969798991001011021031041051061071081091101111121131141151161171181191201211221231241251261271281291301311321331341351361371381391401411421431441451461471481491501511521531541551561571581591601611621631641651661671681691701711721731741751761771781791801811821831841851861871881891901911921931941951961971981992002012022032042052062072082092102112122132142152162172182192202212222232242252262272282292302312322332342352362372382392402412422432442452462472482492502512522532542552562572582592602612622632642652662672682692702712722732742752762772782792802812822832842852862872882892902912922932942952962972982993003013023033043053063073083093103113123133143153163173183193203213223233243253263273283293303313323333343353363373383393403413423433443453463473483493503513523533543553563573583593603613623633643653663673683693

#### Testing Data

In [ ]:
CMYK_Lab = {'C':[],'M':[],'Y':[],'K':[],'L':[],'a':[],'b':[]}

app = Dispatch("PhotoShop.Application")

with Session() as ps: #開新檔案
    ps.app.preferences.rulerUnits = ps.Units.Pixels
    ps.app.documents.add(1920, 1080, name="my_new_document")
    #文字顏色
    doc = ps.active_document
    new_text_layer = doc.artLayers.add()  #新增圖層
    new_text_layer.kind = ps.LayerKind.TextLayer #指定為字圖層
    new_text_layer.textItem.contents = "Hello, World!" #打字
    new_text_layer.textItem.size = 100
    textColor = ps.SolidColor()
    for i in range(100):
        c,m,y,k = random.randint(0,100), random.randint(0,100), random.randint(0,100), random.randint(0,100)
        textColor.cmyk.cyan= c
        textColor.cmyk.magenta= m
        textColor.cmyk.yellow = y
        textColor.cmyk.black = k
        #命令更換文字顏色
        new_text_layer.textItem.color = textColor
        L = round(new_text_layer.textItem.color.Lab.L,0)
        a = round(new_text_layer.textItem.color.Lab.a +0.5,0)
        b = round(new_text_layer.textItem.color.Lab.b +0.5,0)
        CMYK_Lab['C'].append(c), CMYK_Lab['M'].append(m), CMYK_Lab['Y'].append(y)
        CMYK_Lab['K'].append(k)
        CMYK_Lab['L'].append(L), CMYK_Lab['a'].append(a), CMYK_Lab['b'].append(b)
        time.sleep(randint(0,2))
        print(len(CMYK_Lab['C']),end="")
        
CMYK_Lab = pd.DataFrame(CMYK_Lab)
CMYK_Lab.to_csv('TestDataSet.csv',index=False)

In [9]:
"""
為套用Color Profile，需先設立好指定ICC的檔案再開啟
"""
CMYK_Lab = {'C':[],'M':[],'Y':[],'K':[],'L':[],'a':[],'b':[]}

app = GetActiveObject("Photoshop.Application")
file = 'C:\python3.7\Scripts\programs\Fogra51.psd'
app.open(file)

with Session() as ps: 
    #文字顏色
    doc = ps.active_document
    new_text_layer = doc.artLayers.add()  #新增圖層
    new_text_layer.kind = ps.LayerKind.TextLayer #指定為字圖層
    new_text_layer.textItem.contents = "Hello, World!" #打字
    new_text_layer.textItem.size = 100
    textColor = ps.SolidColor()
    for i in range(100):
        c,m,y,k = random.randint(0,100), random.randint(0,100), random.randint(0,100), random.randint(0,100)
        textColor.cmyk.cyan= c
        textColor.cmyk.magenta= m
        textColor.cmyk.yellow = y
        textColor.cmyk.black = k
        #命令更換文字顏色
        new_text_layer.textItem.color = textColor
        L = np.round(new_text_layer.textItem.color.Lab.L,0)
        a = np.round(new_text_layer.textItem.color.Lab.a +0.5,0)
        b = np.round(new_text_layer.textItem.color.Lab.b +0.5,0)
        CMYK_Lab['C'].append(c), CMYK_Lab['M'].append(m), CMYK_Lab['Y'].append(y)
        CMYK_Lab['K'].append(k)
        CMYK_Lab['L'].append(L), CMYK_Lab['a'].append(a), CMYK_Lab['b'].append(b)
        time.sleep(random.randint(0,2))
        print(len(CMYK_Lab['C']),end="")
        
CMYK_Lab = pd.DataFrame(CMYK_Lab)
CMYK_Lab.to_csv('FograTestDataSet3.csv',index=False)

123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100

## Data Loader
<h5>資料處理 : CMYK/100, L/100, a/256, b/256</h>
<h5><font color="red"><B>待處理Lab負值</B></font></h>

In [ ]:
class DataSet():
    def __init__(self,Datas):
        self.data = Datas
    def CMYK2Tensor(self):
        Data = self.data
        CMYKlst=[]
        for i in range(len(Data)):
            CMYKv=torch.Tensor([Data.iloc[i][0]/100,Data.iloc[i][1]/100,
                                   Data.iloc[i][2]/100,Data.iloc[i][3]/100])
            CMYKlst.append(CMYKv)
        CMYK=torch.stack(CMYKlst)
        return CMYK
    def Lab2Tensor(self):
        Data = self.data
        Lablst=[]
        for i in range(len(Data)):
            Labv=torch.Tensor([Data.iloc[i][4]/100,Data.iloc[i][5]/128,
                                   Data.iloc[i][6]/128])
            Lablst.append(Labv)
        Lab=torch.stack(Lablst)
        return Lab

Train=pd.read_csv('FograTrainDataSet.csv',index_col=False)
Test=pd.read_csv('FograTestDataSet.csv',index_col=False)

TrData, TeData = DataSet(Train), DataSet(Test)
TrCMYK, TeCMYK = TrData.CMYK2Tensor(), TeData.CMYK2Tensor()
TrLab, TeLab = TrData.Lab2Tensor(), TeData.Lab2Tensor()

print(TrCMYK.shape,TeCMYK.shape,TrLab.shape,TeLab.shape)

In [ ]:
batch_size = 100
train_set = TensorDataset(TrCMYK,TrLab)
test_set = TensorDataset(TeCMYK,TeLab)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_set, batch_size=batch_size, shuffle=False)

## ANN model

#### Training

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(4, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 32)
        self.fc5 = nn.Linear(32,3)
    def forward(self, x):
        x = self.fc1(x).relu()
        x = self.fc2(x).relu()
        x = self.fc3(x).relu()
        x = self.fc4(x).relu()
        x = self.fc5(x).tanh()
        return x
model = MLP()
print(model)

In [ ]:
epochs = 10000
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.MSELoss()
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
loss_min = 0.005
model.to(device)
model.train()

for epoch in range(epochs):
    loss_total=0
    n=0
    outsave, ysave = [],[]
    for i,data in enumerate(train_loader):
        x, y = data
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        # save batchs out&y results in 1 epoch
        out, y = out.to('cpu'), y.to('cpu')
        outlst = out.detach().numpy().tolist()
        ylst = y.detach().numpy().tolist()
        outsave += outlst
        ysave += ylst
        # Loss
        loss = criterion(out, y)
        loss_total = loss_total + loss
        n = n + 1
        # 反向
        loss.backward()
        # 更新
        optimizer.step()
    loss_ave = loss_total / n
    if (epoch % 100 == 0):
        print('epoch = %8d, loss_ave = %20.12f' % (epoch, loss_ave))
    #儲存最佳loss model
    if loss_ave < loss_min:
        torch.save(model.state_dict(), 'bestmodel.pt')
        print(f'save at epoch = {epoch},loss = {loss_ave}')
        loss_min = loss_ave
        outTrsave, yTrsave = outsave, ysave
    #儲存last epoch model
    if epoch == epochs-1:
        torch.save(model.state_dict(), 'model_last.pt')
        print(f'save last epoch = {epoch},loss = {loss_ave}')

In [ ]:
def LabRecover(data):
    Recover= []
    try:
        for i in range(len(data)):
            Recover.append([data[i][0]*100,data[i][1]*128,data[i][2]*128])
    except:
        Recover.append([data[0]*100,data[1]*128,data[2]*128])
    return Recover

def CMYKRecover(data):
    Recover= []
    try:
        for i in range(len(data)):
            Recover.append([data[i][0]*100,data[i][1]*100,data[i][2]*100,data[i][3]*100])
    except:
        Recover.append([data[0]*100,data[1]*100,data[2]*100,data[3]*100])
    return Recover

def DeltaE_1976(result,label):
    dE = []
    for i in range(len(result)):
        dE.append(((result[i][0]-label[i][0])**2+(result[i][1]-label[i][1])**2+
                       (result[i][2]-label[i][2])**2)**0.5)
    return dE
#Lab值恢復
outTrRe,yTrRe = LabRecover(outTrsave), LabRecover(yTrsave)
#dE計算
dE_Tr = DeltaE_1976(outTrRe,yTrRe)
dE_TrAll, dE_TrAve = sum(dE_Tr), sum(dE_Tr)/1296
dE_TrMax, dE_TrMin = max(dE_Tr), min(dE_Tr)
MaxE_TrIndex, MinE_TrIndex = dE_Tr.index(dE_TrMax), dE_Tr.index(dE_TrMin)
MaxE_TrColor, MinE_TrColor = yTrRe[MaxE_TrIndex], yTrRe[MinE_TrIndex]
MaxE_TrLabOut,MinE_TrLabOut = outTrRe[MaxE_TrIndex], outTrRe[MinE_TrIndex]

print('平均Delta_E:', dE_TrAve)
print('最大Delta_E:', dE_TrMax,'\n最大Delta_ELab:', MaxE_TrColor,'\n最大Delta_E算出Lab:', MaxE_TrLabOut,)
print('最小Delta_E:', dE_TrMin,'\n最小Delta_ELab:', MinE_TrColor,'\n最小Delta_E算出Lab:', MinE_TrLabOut)

### Testing

In [ ]:
model = MLP()
model.load_state_dict(torch.load('bestmodel.pt'))
model.to(device)
model.eval()

for i,data in enumerate(test_loader):
    xte, yte = data
    xte, yte = xte.to(device), yte.to(device)
    outte = model(xte)

outte, yte = outte.to('cpu'), yte.to('cpu')
outte = outte.detach().numpy().tolist()
yte = yte.detach().numpy().tolist()  

In [ ]:
outTeRe,yTeRe = LabRecover(outte), LabRecover(yte)
dE_Te = DeltaE_1976(outTeRe,yTeRe)
dE_TeAll, dE_TeAve = sum(dE_Te), sum(dE_Te)/100
dE_TeMax, dE_TeMin = max(dE_Te), min(dE_Te)
MaxE_TeIndex, MinE_TeIndex = dE_Te.index(dE_TeMax), dE_Te.index(dE_TeMin)
MaxE_TeColor, MinE_TeColor = yTeRe[MaxE_TeIndex], yTeRe[MinE_TeIndex]
MaxE_TeLabOut,MinE_TeLabOut = outTeRe[MaxE_TeIndex], outTeRe[MinE_TeIndex]

print('平均Delta_E:', dE_TeAve)
print('最大Delta_E:', dE_TeMax,'\n最大Delta_E顏色:', MaxE_TeColor,'\n最大Delta_E算出Lab:', MaxE_TeLabOut)
print('最小Delta_E :', dE_TeMin,'\n最小Delta_E顏色:', MinE_TeColor,'\n最小Delta_E算出Lab:', MinE_TeLabOut)

### Other Test Data

In [ ]:
Test2=pd.read_csv('FograTestDataSet2.csv',index_col=False)
TeData2 = DataSet(Test2)
TeCMYK2 = TeData2.CMYK2Tensor()
TeLab2 = TeData2.Lab2Tensor()
test_set2 = TensorDataset(TeCMYK2,TeLab2)
test_loader2  = DataLoader(test_set2, batch_size=batch_size, shuffle=False)

model = MLP()
model.load_state_dict(torch.load('bestmodel.pt'))
model.to(device)
model.eval()

for i,data in enumerate(test_loader2):
    xte2, yte2 = data
    xte2, yte2 = xte2.to(device), yte2.to(device)
    outte2 = model(xte2)

outte2, yte2 = outte2.to('cpu'), yte2.to('cpu')
outte2 = outte2.detach().numpy().tolist()
yte2 = yte2.detach().numpy().tolist()  

outTeRe2,yTeRe2 = LabRecover(outte2), LabRecover(yte2)
dE_Te2 = DeltaE_1976(outTeRe2,yTeRe2)
dE_TeAll2, dE_TeAve2 = sum(dE_Te2), sum(dE_Te2)/100
dE_TeMax2, dE_TeMin2 = max(dE_Te2), min(dE_Te2)
MaxE_TeIndex2, MinE_TeIndex2 = dE_Te2.index(dE_TeMax2), dE_Te2.index(dE_TeMin2)
MaxE_TeColor2, MinE_TeColor2 = yTeRe2[MaxE_TeIndex2], yTeRe2[MinE_TeIndex2]
MaxE_TeLabOut2, MinE_TeLabOut2 = outTeRe2[MaxE_TeIndex2], outTeRe2[MinE_TeIndex2]

print('平均Delta_E:', dE_TeAve2)
print('最大Delta_E:', dE_TeMax2,'\n最大Delta_E顏色:', MaxE_TeColor2,'\n最大Delta_E算出Lab:', MaxE_TeLabOut2)
print('最小Delta_E :', dE_TeMin2,'\n最小Delta_E顏色:', MinE_TeColor2,'\n最小Delta_E算出Lab:', MinE_TeLabOut2)

## 儲存紀錄

In [ ]:
TrRecord = {'Train': dE_TrAve,
          'TRMaxE':dE_TrMax,
          'TRMaxELab:':MaxE_TrColor,
          'TRMaxE_CalLab:':MaxE_TrLabOut,
          'TRMinE:':dE_TrMin,
          'TRMinELab:':MinE_TrColor,
          'TRMinE_CalLab:':MinE_TrLabOut}
TeRecord = {'Test': dE_TeAve,
          'T1MaxE':dE_TeMax,
          'T1MaxELab:':MaxE_TeColor,
          'T1MaxE_CalLab:':MaxE_TeLabOut,
          'T1MinE:':dE_TeMin,
          'T1MinELab:':MinE_TeColor,
          'T1MinE_CalLab:':MinE_TeLabOut}
TeRecord2 = {'Test2': dE_TeAve2,
          'T2MaxE':dE_TeMax2,
          'T2MaxELab:':MaxE_TeColor2,
          'T2MaxE_CalLab:':MaxE_TeLabOut2,
          'T2MinE:':dE_TeMin2,
          'T2MinELab:':MinE_TeColor2,
          'T2MinE_CalLab:':MinE_TeLabOut2}

TrRecord,TeRecord,TeRecord2 = pd.DataFrame(TrRecord),pd.DataFrame(TeRecord),pd.DataFrame(TeRecord2)
Record = TrRecord.join(TeRecord,how='left')
Record = Record.join(TeRecord2,how='left')

Record.to_csv('CMYK2Lab_Record.csv',mode='a',encoding='utf-8-sig')

## 顏色查看